In [1]:
import os

# --- 1. CẤU HÌNH GỐC (ROOT) ---
# Dùng r'' để Python hiểu đây là đường dẫn Windows (không bị lỗi ký tự đặc biệt)
ROOT = r'E:\CAFA-6-Protein-Function-Prediction'

# --- 2. ĐỊNH NGHĨA CÁC THƯ MỤC CON (Tự động nối đuôi) ---
# os.path.join giúp nối đường dẫn đúng chuẩn cho cả Windows/Linux/Mac
INPUT_DIR  = os.path.join(ROOT, 'input')
OUTPUT_DIR = os.path.join(ROOT, 'output')
MODEL_DIR  = os.path.join(ROOT, 'models')

# Tạo sẵn thư mục output và models nếu chưa có (Tránh lỗi không lưu được file)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"✅ Đã cấu hình ROOT: {ROOT}")
print(f"📂 Input: {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"📂 Models: {MODEL_DIR}")

✅ Đã cấu hình ROOT: E:\CAFA-6-Protein-Function-Prediction
📂 Input: E:\CAFA-6-Protein-Function-Prediction\input
📂 Output: E:\CAFA-6-Protein-Function-Prediction\output
📂 Models: E:\CAFA-6-Protein-Function-Prediction\models


In [2]:
import pandas as pd
import numpy as np
import torch
from Bio import SeqIO
from tqdm.auto import tqdm
import gc
import os

# ==========================================
# CẤU HÌNH
# ==========================================
class Config:
    RAW_DATA_DIR = os.path.join(INPUT_DIR, 'cafa-6-protein-function-prediction')
    EMBED_DIR = os.path.join(INPUT_DIR, 'cafa-6-t5-embeddings')
    
    # File Output
    KNN_OUTPUT = os.path.join(OUTPUT_DIR, 'submission_knn.tsv')
    EXACT_OUTPUT = os.path.join(OUTPUT_DIR, 'submission_exact.tsv')

    TRAIN_SEQ = os.path.join(RAW_DATA_DIR, 'Train', 'train_sequences.fasta')
    TRAIN_TERMS = os.path.join(RAW_DATA_DIR, 'Train', 'train_terms.tsv')
    TEST_SEQ = os.path.join(RAW_DATA_DIR, 'Test', 'testsuperset.fasta')
    
    TRAIN_EMBEDS = os.path.join(EMBED_DIR, 'train_embeds.npy')
    TEST_EMBEDS = os.path.join(EMBED_DIR, 'test_embeds.npy')
    
    # Params
    MIN_SCORE = 0.01    # Chỉ lưu các điểm > 0.01 để giảm dung lượng file
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfg = Config()
print(f"Running on: {cfg.DEVICE}")

# ==========================================
# MODULE 1: EXACT MATCH LOOKUP
# ==========================================
def run_exact_match():
    print("\n>>> 1. Running Exact Sequence Matching (Lookup)...")
    
    # 1. Load Train Sequences
    print("   Indexing Train Sequences...")
    train_seq_map = {}
    # Dùng try-except để tránh lỗi nếu file không tồn tại (cho debug)
    if not os.path.exists(cfg.TRAIN_SEQ):
        print(f"Error: Not found {cfg.TRAIN_SEQ}")
        return pd.DataFrame()

    for record in SeqIO.parse(cfg.TRAIN_SEQ, "fasta"):
        seq_str = str(record.seq)
        if '|' in record.id:
            pid = record.id.split('|')[1]
        else:
            pid = record.id
        if seq_str not in train_seq_map:
            train_seq_map[seq_str] = []
        train_seq_map[seq_str].append(pid)
        
    # 2. Load Train Terms
    print("   Loading Train Terms...")
    df_terms = pd.read_csv(cfg.TRAIN_TERMS, sep='\t', names=['id', 'term', 'aspect'])
    df_terms['id'] = df_terms['id'].astype(str).str.strip()
    
    # Lấy Top 5000 terms phổ biến nhất để lookup cho nhanh
    top_terms = set(df_terms['term'].value_counts().head(5000).index)
    df_terms = df_terms[df_terms['term'].isin(top_terms)]
    
    id_to_terms = df_terms.groupby('id')['term'].apply(set).to_dict()
    
    # 3. Match Test Sequences
    print("   Matching Test Sequences...")
    exact_matches = []
    match_count = 0
    
    for record in tqdm(SeqIO.parse(cfg.TEST_SEQ, "fasta"), desc="Exact Matching"):
        test_seq = str(record.seq)
        if '|' in record.id:
            test_id = record.id.split('|')[1]
        else:
            test_id = record.id
        
        if test_seq in train_seq_map:
            train_ids = train_seq_map[test_seq]
            collected_terms = set()
            for tid in train_ids:
                if tid in id_to_terms:
                    collected_terms.update(id_to_terms[tid])
            
            if collected_terms:
                match_count += 1
                for term in collected_terms:
                    # Gán điểm 0.99 (gần tuyệt đối)
                    exact_matches.append({'id': test_id, 'term': term, 'exact_score': 0.99})
                
    print(f"   >>> FOUND {match_count} EXACT MATCHES!")
    return pd.DataFrame(exact_matches)

# ==========================================
# MODULE 2: WEIGHTED KNN (MEMORY OPTIMIZED)
# ==========================================
def run_knn():
    print("\n>>> 2. Running Weighted KNN (Streaming Mode)...")
    
    # Load Data
    print("   Loading embeddings...")
    X_train = np.load(cfg.TRAIN_EMBEDS)
    X_test = np.load(cfg.TEST_EMBEDS)
    
    # Chuyển sang Torch & Normalize
    X_train = torch.from_numpy(X_train).to(cfg.DEVICE)
    X_train = torch.nn.functional.normalize(X_train, p=2, dim=1)
    
    X_test = torch.from_numpy(X_test) # Chưa đẩy lên GPU vội
    X_test = torch.nn.functional.normalize(X_test, p=2, dim=1)
    
    # Load Labels Map
    print("   Loading ID mapping...")
    df = pd.read_csv(cfg.TRAIN_TERMS, sep='\t', names=['id', 'term', 'aspect'])
    
    # Load Train IDs
    train_ids_npy = np.load(os.path.join(cfg.EMBED_DIR, 'train_ids.npy'))
    train_ids_npy = [str(x).split('|')[1] if '|' in str(x) else str(x) for x in train_ids_npy]
    
    # Tạo Map: Index -> List of Terms
    id_to_terms = df.groupby('id')['term'].apply(list).to_dict()
    idx_to_terms = {}
    for i, pid in enumerate(train_ids_npy):
        if pid in id_to_terms:
            idx_to_terms[i] = id_to_terms[pid]
            
    # Load Test IDs
    test_ids_npy = np.load(os.path.join(cfg.EMBED_DIR, 'test_ids.npy'))
    test_ids_npy = [str(x).split('|')[1] if '|' in str(x) else str(x) for x in test_ids_npy]
    
    # Config KNN
    K = 10
    BATCH_SIZE = 500 # Giữ batch nhỏ để tiết kiệm VRAM
    num_test = len(test_ids_npy)
    
    print(f"   Writing results directly to {cfg.KNN_OUTPUT}...")
    
    # --- [SỬA LỖI LOGIC QUAN TRỌNG Ở ĐÂY] ---
    # Mở file TRƯỚC, sau đó loop tính toán BÊN TRONG
    with open(cfg.KNN_OUTPUT, 'w') as f:
        # Header (Optional, nhưng thường submission không cần header)
        # f.write("ProteinID\tGO_ID\tScore\n") 
        
        for i in tqdm(range(0, num_test, BATCH_SIZE), desc="KNN Computing"):
            # 1. Lấy batch test & đẩy lên GPU
            batch_test = X_test[i : i + BATCH_SIZE].to(cfg.DEVICE)
            
            # 2. Tính Cosine Similarity (Matrix Multiplication)
            sim_matrix = torch.mm(batch_test, X_train.t())
            
            # 3. Lấy Top K
            vals, inds = torch.topk(sim_matrix, k=K, dim=1)
            vals = vals.cpu().numpy()
            inds = inds.cpu().numpy()
            
            # 4. Xử lý từng protein trong batch
            for j in range(len(batch_test)):
                tid = test_ids_npy[i + j]
                scores = {}
                total_weight = 0
                
                # Cộng dồn điểm từ K hàng xóm
                for k in range(K):
                    idx = inds[j, k]
                    sim = vals[j, k]
                    
                    if sim > 0.3: # Chỉ xét hàng xóm có độ tương đồng > 0.3
                        weight = sim * sim
                        total_weight += weight
                        if idx in idx_to_terms:
                            for term in idx_to_terms[idx]:
                                scores[term] = scores.get(term, 0) + weight
                                
                # Chuẩn hóa điểm số
                if total_weight > 0:
                    for term in scores:
                        scores[term] /= total_weight
                        scores[term] *= min(1.0, total_weight)
                
                # 5. GHI FILE NGAY LẬP TỨC (Streaming)
                for term, score in scores.items():
                    if score > cfg.MIN_SCORE:
                        f.write(f"{tid}\t{term}\t{score:.3f}\n")
                        
            # Giải phóng VRAM sau mỗi batch
            del batch_test, sim_matrix, vals, inds
            torch.cuda.empty_cache()

    # Dọn dẹp cuối cùng
    del X_train, X_test
    gc.collect()
    print("   KNN Done!")

# ==========================================
# MAIN EXECUTION
# ==========================================
def main():
    # 1. Chạy Exact Match
    df_exact = run_exact_match()
    print(f">>> Saving Exact Match to {cfg.EXACT_OUTPUT}...")
    if not df_exact.empty:
        df_exact[['id', 'term', 'exact_score']].to_csv(cfg.EXACT_OUTPUT, sep='\t', index=False, header=False)
    else:
        print("Warning: No exact matches found. File will be empty.")
        # Tạo file rỗng để Notebook 05 không bị lỗi file not found
        open(cfg.EXACT_OUTPUT, 'w').close() 
    
    # 2. Chạy KNN (Hàm này đã tự save file submission_knn.tsv)
    run_knn()
    
    print("\nSUCCESS! Generated 2 files:")
    print(f"1. {cfg.EXACT_OUTPUT}")
    print(f"2. {cfg.KNN_OUTPUT}")
    print("Now proceed to Notebook 05 to ensemble these files.")

if __name__ == "__main__":
    main()

Running on: cuda

>>> 1. Running Exact Sequence Matching (Lookup)...
   Indexing Train Sequences...
   Loading Train Terms...
   Matching Test Sequences...


Exact Matching: 0it [00:00, ?it/s]

   >>> FOUND 84562 EXACT MATCHES!
>>> Saving Exact Match to E:\CAFA-6-Protein-Function-Prediction\output\submission_exact.tsv...

>>> 2. Running Weighted KNN (Streaming Mode)...
   Loading embeddings...
   Loading ID mapping...
   Writing results directly to E:\CAFA-6-Protein-Function-Prediction\output\submission_knn.tsv...


KNN Computing:   0%|          | 0/449 [00:00<?, ?it/s]

   KNN Done!

SUCCESS! Generated 2 files:
1. E:\CAFA-6-Protein-Function-Prediction\output\submission_exact.tsv
2. E:\CAFA-6-Protein-Function-Prediction\output\submission_knn.tsv
Now proceed to Notebook 05 to ensemble these files.
